# Try this with Dask First

As with Question 3 I'm going to try this firstly with Dask and then with Spark. 

In [1]:
#just finding my exact file name here...
!ls

03_test.ipynb	       fhvhv_tripdata_2021-01.csv
04_pyspark.ipynb       fhvhv_tripdata_2021-01.csv.1
05_taxi_schema.ipynb   fhvhv_tripdata_2021-02.csv
06_spark_sql.ipynb     fhvhv_tripdata_2021-02_homeworkQ3.ipynb
07_groupby_join.ipynb  fhvhv_tripdata_2021-02_homeworkQ4.ipynb
08_rdds.ipynb	       head.csv
data		       HomeworkQ3.ipynb
download_data.sh       test.py
fhvhv


In [31]:
import dask.dataframe as dd

In [32]:
filename = 'fhvhv_tripdata_2021-02.csv'
df = dd.read_csv(filename, parse_dates=['pickup_datetime', 'dropoff_datetime'])

In [33]:
df['trip_duration'] = df['dropoff_datetime'] - df['pickup_datetime']
#print(df.compute()) # this step took around 5 minutes...

In [34]:
df.dtypes

hvfhs_license_num                object
dispatching_base_num             object
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
PULocationID                      int64
DOLocationID                      int64
SR_Flag                         float64
trip_duration           timedelta64[ns]
dtype: object

In [35]:
max_trip_time = df['trip_duration'].max()

In [36]:
max_trip_time.compute()

Timedelta('0 days 20:59:00')

# Answer in Dask

In [37]:
longest_trips = df[(df.trip_duration == max_trip_time)]
longest_trips.compute()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,trip_duration
139863,HV0005,B02510,2021-02-11 13:40:44,2021-02-12 10:39:44,247,41,NaN,0 days 20:59:00


# Now try with pyspark

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('fhvhv_tripdata_2021-02_homework') \
    .getOrCreate()

22/03/01 10:15:15 WARN Utils: Your hostname, sandy-ODYSSEY-X86J4105 resolves to a loopback address: 127.0.1.1; using 10.0.0.1 instead (on interface wlo2)
22/03/01 10:15:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/01 10:15:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
#!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

In [7]:
#!wc -l fhvhv_tripdata_2021-02.csv

In [8]:
from pyspark.sql import types

In [9]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

I don't need to bother repartitioning etc. here as I've already done it before. I'll turn these code cells into markdown cells and simply start off from reading the parquet file

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

df = df.repartition(24)

df.write.parquet('fhvhv/2021/02/')

In [10]:
df = spark.read.parquet('fhvhv/2021/02/')

In [11]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [22]:
# create a new column - see https://sparkbyexamples.com/pyspark/pyspark-timestamp-difference-seconds-minutes-hours/
# get the difference in seconds using unix_timestamp
from pyspark.sql.functions import *

df_trip_length = df \
    .withColumn('trip_length_sec',
                 unix_timestamp('dropoff_datetime') - unix_timestamp('pickup_datetime'))

In [14]:
df_trip_length.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- trip_length_sec: long (nullable = true)



# Answer in Pyspark

In [18]:
# Register the DataFrame as a SQL temporary view
df_trip_length.createOrReplaceTempView("fhvhv_2021_02_trip_duration")

sqlDF = spark.sql("SELECT hvfhs_license_num, dispatching_base_num, \
                          pickup_datetime, dropoff_datetime, trip_length_sec \
                     FROM fhvhv_2021_02_trip_duration \
                    ORDER BY trip_length_sec DESC")
sqlDF.show()

+-----------------+--------------------+-------------------+-------------------+---------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|trip_length_sec|
+-----------------+--------------------+-------------------+-------------------+---------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|          75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|          57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|          44039|
|           HV0003|              B02864|2021-02-03 20:24:25|2021-02-04 07:41:58|          40653|
|           HV0003|              B02887|2021-02-19 23:17:44|2021-02-20 09:44:01|          37577|
|           HV0003|              B02764|2021-02-25 17:13:35|2021-02-26 02:57:05|          35010|
|           HV0003|              B02875|2021-02-20 01:36:13|2021-02-20 11:16:19|          34806|
|           HV0005|           

In [19]:
# Getting just the max time
sqlDF = spark.sql("SELECT MAX(trip_length_sec) \
                     FROM fhvhv_2021_02_trip_duration")
sqlDF.show()

+--------------------+
|max(trip_length_sec)|
+--------------------+
|               75540|
+--------------------+



In [21]:
# Combine the above into another way of finding out

sqlDF = spark.sql("SELECT hvfhs_license_num, dispatching_base_num, \
                          pickup_datetime, dropoff_datetime, trip_length_sec \
                     FROM fhvhv_2021_02_trip_duration \
                    WHERE trip_length_sec == ( \
                        SELECT MAX(trip_length_sec) \
                          FROM fhvhv_2021_02_trip_duration) \
                  ")
sqlDF.show()

+-----------------+--------------------+-------------------+-------------------+---------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|trip_length_sec|
+-----------------+--------------------+-------------------+-------------------+---------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|          75540|
+-----------------+--------------------+-------------------+-------------------+---------------+

